Augmentation


In [13]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from tqdm import tqdm
import random



def add_salt_and_pepper_noise(image, amount=0.04, salt_vs_pepper=0.5):
    """Add salt-and-pepper noise to an image."""
    image = image.convert('RGB')
    img_array = np.array(image)
    h, w = img_array.shape[:2]
    num_salt = int(amount * h * w * salt_vs_pepper)
    num_pepper = int(amount * h * w * (1 - salt_vs_pepper))

    # Add salt (white)
    coords = [np.random.randint(0, i - 1, num_salt) for i in img_array.shape[:2]]
    img_array[coords[0], coords[1]] = 255

    # Add pepper (black)
    coords = [np.random.randint(0, i - 1, num_pepper) for i in img_array.shape[:2]]
    img_array[coords[0], coords[1]] = 0

    return Image.fromarray(img_array)

def add_gaussian_noise(image, mean=0, std=0.05):
    """Add Gaussian noise to an image."""
    image = image.convert('RGB')
    img_array = np.array(image) / 255.0  # Normalize to [0, 1]
    noise = np.random.normal(mean, std, img_array.shape)
    noisy_img_array = img_array + noise
    noisy_img_array = np.clip(noisy_img_array, 0, 1)  # Ensure values are between 0 and 1
    return Image.fromarray((noisy_img_array * 255).astype(np.uint8))

def random_zoom(image, crop_percentage=0.1):
    """Randomly zoom into an image by cropping a percentage from each side."""
    width, height = image.size
    
    # Calculate the cropping area
    left = int(width * crop_percentage)
    top = int(height * crop_percentage)
    right = width - int(width * crop_percentage)
    bottom = height - int(height * crop_percentage)
    
    # Crop the image
    cropped_img = image.crop((left, top, right, bottom))
    
    # Resize back to original size
    return cropped_img.resize((width, height), Image.Resampling.LANCZOS)


def adjust_brightness_contrast(image, brightness_factor=1.2, contrast_factor=1.2):
    """Adjust the brightness and contrast of an image."""
    image = image.convert("RGB")  # Ensure the image is in RGB mode
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)
    
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)
    
    return image

def add_gaussian_blur(image, radius=2):
    # Ensure the input is a valid image
    if not isinstance(image, Image.Image):
        raise ValueError("The input is not a valid PIL Image object.")
    
    # Convert non-RGB modes to RGB
    if image.mode not in ('RGB', 'RGBA'):
        image = image.convert('RGB')
    
    # Apply Gaussian blur
    return image.filter(ImageFilter.GaussianBlur(radius))

def darken(image, factor=0.6):
    # Ensure image is in RGB mode
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(factor)



MAX_CLASS_SIZE = 500

def apply_augmentation_to_class(class_images, class_size, class_name):
    augmented_images = []
    
    if class_size >= MAX_CLASS_SIZE:
        print(f"Skipping augmentation for {class_name} as class size has reached the limit of {MAX_CLASS_SIZE}.")
        return augmented_images  # No augmentation for this class

    for img_info in class_images:
        img, original_image_name = img_info
        count = 0 
        if class_size >= MAX_CLASS_SIZE:
            print(f"Reached max class size for {class_name}, stopping augmentation.")
            break  # Stop augmentation if class size reaches the limit
        
        # original_image_name = os.path.splitext(os.path.basename(img.filename))[0]

        if class_size < 50:
            # Apply all augmentations and combinations of two
            augmented_images.append((add_salt_and_pepper_noise(img), f'{original_image_name}_salt_and_pepper_noise'))
            augmented_images.append((add_gaussian_noise(img), f'{original_image_name}_gaussian_noise'))
            augmented_images.append((random_zoom(img), f'{original_image_name}_random_zoom'))
            augmented_images.append((adjust_brightness_contrast(img), f'{original_image_name}_brightness_contrast'))

            # Convert to RGB before applying Gaussian blur
            img_rgb = img.convert("RGB")
            augmented_images.append((img_rgb.filter(ImageFilter.GaussianBlur(radius=2)), f'{original_image_name}_gaussian_blur'))

            augmented_images.append((darken(img), f'{original_image_name}_darken'))  # Darken the image

            # Apply combinations of two augmentations
            augmented_images.append((add_gaussian_noise(random_zoom(img)), f'{original_image_name}_gaussian_noise_zoom'))
            augmented_images.append((add_salt_and_pepper_noise(adjust_brightness_contrast(img)), f'{original_image_name}_salt_brightness_contrast'))


        elif 50 <= class_size < 100:
            # Apply all augmentations and combinations of two
            augmented_images.append((add_salt_and_pepper_noise(img), f'{original_image_name}_salt_and_pepper_noise'))
            augmented_images.append((add_gaussian_noise(img), f'{original_image_name}_gaussian_noise'))
            
            augmented_images.append((adjust_brightness_contrast(img), f'{original_image_name}_brightness_contrast'))
            augmented_images.append((img.filter(ImageFilter.GaussianBlur(radius=2)), f'{original_image_name}_gaussian_blur'))
        
        
            
        elif class_size>= 100:
            augmented_images.append((add_salt_and_pepper_noise(img), f'{original_image_name}_salt_and_pepper_noise'))
            augmented_images.append((add_gaussian_noise(img), f'{original_image_name}_gaussian_noise'))
            augmented_images.append((img.filter(ImageFilter.GaussianBlur(radius=2)), f'{original_image_name}_gaussian_blur'))
            augmented_images.append((adjust_brightness_contrast(img), f'{original_image_name}_brightness_contrast'))
            class_size += 4
            if class_size >= MAX_CLASS_SIZE: break
        
        '''if class_size >= MAX_CLASS_SIZE:
            print(f"Reached max class size for {class_name}, stopping augmentation.")
            break'''
        
    return augmented_images

def save_augmented_images(class_images, augmented_images, class_path):
    for img, img_name in augmented_images:
        # Convert image to RGB before saving if not already in RGB mode
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        img.save(f"{class_path}/{img_name}.jpg", 'JPEG')
# Example usage
def process_images_in_dataset(train_dir):
    class_names = os.listdir(train_dir)
    
    for class_name in class_names:
        class_path = os.path.join(train_dir, class_name)
        
        if os.path.isdir(class_path):
            # Include both Image objects and their original names as tuples
            class_images = [
                (Image.open(os.path.join(class_path, img)).convert('RGB'), os.path.splitext(img)[0])
                for img in os.listdir(class_path) 
                if img.lower().endswith(('png', 'jpg', 'jpeg'))
            ]
            class_size = len(class_images)
            # Apply augmentations based on class size
            augmented_images = apply_augmentation_to_class(class_images, class_size, class_name)
            
            # Save augmented images
            save_augmented_images(class_images, augmented_images, class_path)
            
            print(f"Processed {class_name} with {class_size} original images.")

# Run the processing
train_dir = r"C:\Users\eliza\OneDrive\Desktop\4-cu kurs\FIRST SEM\AI\project\for_test\train"
process_images_in_dataset(train_dir)

Processed African Violet (Saintpaulia ionantha) with 231 original images.
Processed Aloe Vera with 171 original images.
Processed Anthurium (Anthurium andraeanum) with 316 original images.
Processed Areca Palm (Dypsis lutescens) with 132 original images.
Processed Asparagus Fern (Asparagus setaceus) with 115 original images.
Processed Begonia (Begonia spp.) with 158 original images.
Processed Bird of Paradise (Strelitzia reginae) with 123 original images.
Processed Birds Nest Fern (Asplenium nidus) with 203 original images.
Processed Boston Fern (Nephrolepis exaltata) with 208 original images.
Processed Calathea with 230 original images.
Processed Cast Iron Plant (Aspidistra elatior) with 186 original images.
Processed Chinese evergreen (Aglaonema) with 358 original images.
Processed Chinese Money Plant (Pilea peperomioides) with 265 original images.
Processed Christmas Cactus (Schlumbergera bridgesii) with 211 original images.
Processed Chrysanthemum with 145 original images.
Processe